In [2]:
from data_loader import *
from index_Trend import *
from strategy import *

In [82]:
df = pd.read_csv('./Dataset/stock_minute.csv')
df = df.drop(df.columns[0], axis = 1 )

In [83]:
df

,Datetime,Open,High,Low,Close,Adj Close,Volume,Ticker,TickerName
0,2022-10-05 09:30:00-04:00,1.7201,1.7201,1.7201,1.7201,1.7201,3187.0,AACG,ATA Creativity Global American Depositary Shares
1,2022-10-05 11:37:00-04:00,1.8450,1.8500,1.8400,1.8400,1.8400,501.0,AACG,ATA Creativity Global American Depositary Shares
2,2022-10-05 11:55:00-04:00,1.8350,1.8350,1.8350,1.8350,1.8350,301.0,AACG,ATA Creativity Global American Depositary Shares
3,2022-10-05 12:08:00-04:00,1.8475,1.8475,1.8475,1.8475,1.8475,201.0,AACG,ATA Creativity Global American Depositary Shares
4,2022-10-05 12:42:00-04:00,1.8300,1.8300,1.8300,1.8300,1.8300,151.0,AACG,ATA Creativity Global American Depositary Shares
...,...,...,...,...,...,...,...,...,...
4698090,2022-10-17 10:51:00-04:00,8.9400,8.9400,8.9000,8.9000,8.9000,1590.0,ZYXI,Zynex Inc. Common Stock
4698091,2022-10-17 10:52:00-04:00,8.8900,8.9300,8.8800,8.8950,8.8950,9911.0,ZYXI,Zynex Inc. Common Stock
4698092,2022-10-17 10:53:00-04:00,8.9500,8.9500,8.9100,8.9100,8.9100,807.0,ZYXI,Zynex Inc. Common Stock
4698093,2022-10-17 10:58:00-04:00,8.9473,8.9473,8.9473,8.9473,8.9473,724.0,ZYXI,Zynex Inc. Common Stock


In [84]:
df['Datetime'] = to_datetime(df['Datetime'])

In [85]:
goog_df = df.groupby('Ticker').get_group('GOOG')

In [86]:
goog_rsi = rsi(df.groupby('Ticker').get_group('GOOG'))

In [92]:
goog_rsi.shape

(3309, 3)

In [107]:
behavior1 = goog_rsi['RSI'].apply(lambda x : rsi_strategy(x, 70, 30))

In [98]:
money = 30000
stock = 0
m_list = []
s_list = []
for idx, trade in enumerate(behavior):
    if trade == 1 and money > goog_df.iloc[idx]['Adj Close']:
        money -= goog_df.iloc[idx]['Adj Close']
        stock +=1
    elif trade == -1 and stock >0:
        money += goog_df.iloc[idx]['Adj Close']
        stock -=1
    else:
        pass

    m_list.append(money)
    s_list.append(stock)



In [99]:
(s_list[-1]* goog_df.iloc[-1]['Adj Close'] + m_list[-1] - m_list[0])/m_list[0]

0.008237623850504557

In [100]:
goog_boll = bollinger(goog_df,k=2 , w = 2)

In [101]:
goog_boll

,Datetime,Ticker,center,ub,lb
1043725,2022-10-05 09:30:00,GOOG,NaN,NaN,NaN
1043726,2022-10-05 09:31:00,GOOG,100.847500,100.896992,100.798008
1043727,2022-10-05 09:32:00,GOOG,100.892101,101.067745,100.716457
1043728,2022-10-05 09:33:00,GOOG,101.067101,101.386429,100.747772
1043729,2022-10-05 09:34:00,GOOG,101.215000,101.313995,101.116006
...,...,...,...,...,...
3488168,2022-10-17 10:55:00,GOOG,100.831150,100.912752,100.749548
3488169,2022-10-17 10:56:00,GOOG,100.825001,100.923995,100.726006
3488170,2022-10-17 10:57:00,GOOG,100.810001,100.866571,100.753432
3488171,2022-10-17 10:59:00,GOOG,100.895000,101.078844,100.711157


In [102]:
goog_boll[['ub', 'lb']]

,ub,lb
1043725,NaN,NaN
1043726,100.896992,100.798008
1043727,101.067745,100.716457
1043728,101.386429,100.747772
1043729,101.313995,101.116006
...,...,...
3488168,100.912752,100.749548
3488169,100.923995,100.726006
3488170,100.866571,100.753432
3488171,101.078844,100.711157


In [108]:
behavior2 = []
for i in range(len(goog_df)):
    behavior2.append(bollinger_strategy(goog_df.iloc[idx]['Adj Close'], goog_boll.iloc[idx]['ub'], goog_boll.iloc[idx]['lb']))

In [104]:
pd.DataFrame(behavior).value_counts()

0    3309
dtype: int64

In [105]:
money = 30000
stock = 0
m_list = []
s_list = []
for idx, trade in enumerate(behavior):
    if trade == 1 and money > goog_df.iloc[idx]['Adj Close']:
        money -= goog_df.iloc[idx]['Adj Close']
        stock +=1
    elif trade == -1 and stock >0:
        money += goog_df.iloc[idx]['Adj Close']
        stock -=1
    else:
        pass
    m_list.append(money)
    s_list.append(stock)

In [106]:
s_list[-1]* goog_df.iloc[-1]['Adj Close'] + m_list[-1]

30000.0

In [109]:
behavior3 = (np.array(behavior1)+ np.array(behavior2)).tolist()

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
money = 30000
stock = 0
m_list = []
s_list = []
for idx, trade in enumerate(behavior):
    if trade == 1 and money > goog_df.iloc[idx]['Adj Close']:
        money -= goog_df.iloc[idx]['Adj Close']
        stock +=1
    elif trade == -1 and stock >0:
        money += goog_df.iloc[idx]['Adj Close']
        stock -=1
    else:
        pass
    m_list.append(money)
    s_list.append(stock)